## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```


In [ ]:
#%pip install pygis

In [1]:
import ee
import geemap
import leafmap
#https://medium.com/@geonextgis/getting-started-with-geemap-a-guide-to-working-with-feature-and-image-collections-dfa438d59621

In [2]:
geemap.ee_initialize()

## Load data

In [40]:
WorldPop = ee.ImageCollection("WorldPop/GP/100m/pop")
pop_sahel = ee.Image("projects/ee-aboubacarhema94/assets/ACLED/Sahel_pop_2020_UNadj_constrained")
sahel_adm2 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/ACLED/G5_Sahel_adm2")
events_diffusion = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/ACLED/G5_Sahel_Diffusion")
geometry = ee.Geometry.Polygon(
        [[[-19.7019400534855, 29.93665627910383],
          [-19.7019400534855, 5.813630636937255],
          [26.967981821514496, 5.813630636937255],
          [26.967981821514496, 29.93665627910383]]])

pop_sahel = ee.Image(pop_sahel)

In [13]:
viz_pop = {
  'min': 0.0,
  'max': 250.0,
  'palette': ['24126c', '1fff4f', 'd4ff50'],
}

In [120]:
#Extract the projection before doing any computation
projection = pop_sahel.projection()
#print(ee.Number(projection.nominalScale()))

In [41]:
Map = geemap.Map()
# Creat an interactive map
#Map = Map.addLayer(WorldPop, viz_pop, 'Population Sahel', 0);
Map.addLayer(pop_sahel, viz_pop, 'Population Sahel', True, 1)
Map.addLayer(sahel_adm2, {'color': 'purple'}, "sahel adm2")
Map.centerObject(sahel_adm2)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

No such comm: 143f7e26baeb48e69d7aad966de92e9b
No such comm: c719c26bc38c475481563244c318131f


In [96]:

# Compute sums of the specified properties.
pop_sahel_stats = 'pop_sahel_stats.csv'
geemap.zonal_stats(
    pop_sahel,
    sahel_adm2,
    pop_sahel_stats,
    statistics_type='SUM',
    scale=100, 
    return_fc=False
)


Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\AHema\OneDrive - CGIAR\Desktop\2023\NORAD\Political Violence in the G5 Sahel Countries\Conflict diffusion indicator\scripts\pop_sahel_stats.csv


## Start conflict diffusion indicator computation

In [44]:

# year of event
year = 2018

#
pop_seuil = 50

#
nb_events = 3

1. Create a spatial grid of 10km-by-10km

In [127]:
# ****** Aggregation to 10km ****** #
country_pop = pop_sahel

# Get the projection at required scale
projectionAt1k = projection.atScale(1000)
projectionAt10k = projection.atScale(10000)

# Step1: 100m to 1000m
country_popAt1k = country_pop \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  }) \
  .reproject(**{
    'crs': projectionAt1k
  })
# Step2: 1000m to 10000m
country_popAt10k = country_popAt1k \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  }) \
  .reproject(**{
    'crs': projectionAt10k
  })

In [131]:
Map.addLayer(country_popAt1k.clip(sahel_adm2), viz_pop, 'Population Sahel 1km', 1)
Map.addLayer(country_popAt10k.clip(sahel_adm2), viz_pop, 'Population Sahel 10km', 1)
Map

Map(bottom=242559.0, center=[13.526516622348206, -2.808036804199219], controls=(WidgetControl(options=['positi…

2. Remove grid cells with less than 100 habitants using population data from WorldPop (pop_country)

3. Identify grid cells with at least 10 conflict events per year using events_diffusion layer

3.1 Number of Conflict event

In [56]:
import geemap.chart as chart

features = events_diffusion.select('event_type')
df = geemap.ee_to_df(features, sort_columns=True)
df

,event_type
0,Battles
1,Explosions/Remote violence
2,Battles
3,Battles
4,Battles
...,...
15232,Explosions/Remote violence
15233,Violence against civilians
15234,Violence against civilians
15235,Explosions/Remote violence


In [104]:
events_diffusion_country_year = events_diffusion \
                                              .filter(ee.Filter.eq('year', year))

#calculating the number of non-null values
#print('Number of Conflict event',
#      events_diffusion_country_year.aggregate_count('event_type'));
      
#print(events_diffusion_country_year.aggregate_count('event_type'))

3.2 Vizualise Conflict event layer

In [49]:
# Use style() to visualize the points
#events_diffusion_country_year
eventsStyled = {
    'color': 'red',
    'pointSize': 1,
    'pointShape': 'triangle',
    'width': 3,
}

Map.addLayer(events_diffusion_country_year.style(**eventsStyled), {}, "conflict events")
Map

Map(bottom=3935.0, center=[19.84939395842279, -0.5053710937500001], controls=(WidgetControl(options=['position…

No such comm: 143f7e26baeb48e69d7aad966de92e9b
No such comm: c719c26bc38c475481563244c318131f


3.3 Create image from conflict events layer



In [115]:
#/ add dummy property to use for reduceToImage

def func_dlg(feature):
  return feature.set('dummy',1)

if_events = ee.FeatureCollection(events_diffusion_country_year).map(func_dlg)

#check
#print('Check Number of Conflict event',
#      if_events.aggregate_count('dummy'))

In [114]:
eventsImg = if_events.reduceToImage(['dummy'], ee.Reducer.sum().unweighted()) \
  .unmask(0) \
  .reproject('epsg:4326', None, 10000) \
  .clip(geometry)
#print(eventsImg)
#check
eventsImgSum = eventsImg.reduceRegion(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'geometry': geometry,
    'scale': 10000,
    'maxPixels': 1e20
    })

#print(eventsImgSum,'Check Number of Conflict event')

In [118]:
events_sahel_stats = 'events_sahel_stats.csv'
geemap.zonal_stats(
    eventsImg,
    sahel_adm2,
    events_sahel_stats,
    statistics_type='SUM',
    scale=10000, 
    return_fc=False
)

The input zone data must be an ee.FeatureCollection.


In [128]:
snippet = """
Map.addLayer(country_popAt1k.clip(sahel_adm2), viz_pop, 'Population Sahel 1km', 0);
Map.addLayer(country_popAt10k.clip(sahel_adm2), viz_pop, 'Population Sahel 10km', 0);

"""

geemap.js_snippet_to_py(snippet, add_new_cell=True, import_ee=False)


In [ ]:
m.addLayer(country_popAt1k.clip(sahel_adm2), viz_pop, 'Population Sahel 1km', 0)
m.addLayer(country_popAt10k.clip(sahel_adm2), viz_pop, 'Population Sahel 10km', 0)

m

In [126]:
# ****** Aggregation to 10km ****** #

# Get the projection at required scale
projectionAt1k = projection.atScale(1000)
projectionAt10k = projection.atScale(10000)

# Step1: 100m to 1000m
country_popAt1k = country_pop \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  }) \
  .reproject(**{
    'crs': projectionAt1k
  })
# Step2: 1000m to 10000m
country_popAt10k = country_popAt1k \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  }) \
  .reproject(**{
    'crs': projectionAt10k
  })
m

NameError: name 'country_pop' is not defined

In [ ]:
# ****** Aggregation to 10km ****** #

# Get the projection at required scale
projectionAt1k = projection.atScale(1000)
projectionAt10k = projection.atScale(10000)

# Step1: 100m to 1000m
country_popAt1k = country_pop \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  })
  # Request the data at the scale and projection \
  .reproject(**{
    'crs': projectionAt1k
  })
# Step2: 1000m to 10000m
country_popAt10k = country_popAt1k \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  })
  # Request the data at the scale and projection \
  .reproject(**{
    'crs': projectionAt10k
  })
m

In [ ]:

m